In [2]:
import pandas as pd
import numpy as np

# 원본 파일 경로와 저장할 파일 경로 설정
original_file_path = r"C:\Users\EL040\Desktop\Project\data\labeling_sheet.csv"
output_file_path = r"C:\Users\EL040\Desktop\Project\data\labeling_sheet_labeled_final.csv"

# --- 데이터 처리 시작 ---

try:
    # 1. 원본 CSV 파일을 DataFrame으로 불러오기
    df = pd.read_csv(original_file_path)
    print("✅ 원본 파일을 성공적으로 불러왔습니다.")
    
    # 'label' 컬럼이 없는 경우를 대비하여 생성
    if 'label' not in df.columns:
        df['label'] = 0

    # 2. 'label' 컬럼을 0 (정상)으로 초기화 (모든 행을 새로 계산하기 위함)
    df['label'] = 0
    print("-> 'label' 컬럼을 기본값 0으로 초기화합니다.")

    # 3. 규칙 #1: '이상점수' 기반 라벨링
    # 이상점수가 0.635 이상이면 'label'을 1(비정상)로 설정
    anomaly_score_threshold = 0.635
    df.loc[df['이상점수'] >= anomaly_score_threshold, 'label'] = 1
    print(f"-> 규칙 #1 ('이상점수' >= {anomaly_score_threshold}): {df['label'].sum()}개 라벨링 완료.")

    # 4. 규칙 #2: '버킷' 데이터 통계 기반 라벨링
    # 각 버킷 컬럼에서 평균으로부터 크게 벗어나는 값(이상치)을 1(비정상)로 설정
    bucket_columns = ['액터 버킷', '레포 버킷', '조직 버킷']
    std_multiplier = 1.5  # 평균에서 표준편차의 몇 배까지를 정상 범위로 볼 것인지 설정 (값↑: 둔감, 값↓: 민감)

    print(f"-> 규칙 #2 (버킷 이상치, 정상범위: 평균 ± {std_multiplier} * 표준편차):")
    for col in bucket_columns:
        # 평균(mean)과 표준편차(std) 계산
        mean = df[col].mean()
        std = df[col].std()

        # 정상 데이터 범위 설정
        lower_bound = mean - std_multiplier * std
        upper_bound = mean + std_multiplier * std
        
        # 정상 범위를 벗어나는 행을 찾아 'label'을 1로 설정
        # df['label']에 덮어쓰므로, 하나라도 이상치이면 1이 됨
        df.loc[(df[col] < lower_bound) | (df[col] > upper_bound), 'label'] = 1
        
        print(f"   - '{col}' 처리 완료. 현재까지 총 {df['label'].sum()}개 라벨링.")

    # 5. 최종 결과 저장
    # 원본과 다른 이름으로 같은 위치에 저장
    df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    print("\n--- 작업 완료 ---")
    print(f"✅ 모든 라벨링 규칙이 적용된 결과가 아래 경로에 저장되었습니다.")
    print(f"   {output_file_path}")
    
    # 최종 라벨링 결과 요약 출력
    print("\n[최종 'label' 컬럼 요약]")
    print(f"  - 0 (정상): {df['label'].value_counts().get(0, 0)}개")
    print(f"  - 1 (비정상): {df['label'].value_counts().get(1, 0)}개")

except FileNotFoundError:
    print(f"🚨 오류: '{original_file_path}' 경로에 파일이 없습니다. 파일 경로를 다시 확인해주세요.")
except KeyError as e:
    print(f"🚨 오류: CSV 파일에 필요한 컬럼({e})이 없습니다. 컬럼명을 확인해주세요.")

✅ 원본 파일을 성공적으로 불러왔습니다.
-> 'label' 컬럼을 기본값 0으로 초기화합니다.
-> 규칙 #1 ('이상점수' >= 0.635): 2500개 라벨링 완료.
-> 규칙 #2 (버킷 이상치, 정상범위: 평균 ± 1.5 * 표준편차):
   - '액터 버킷' 처리 완료. 현재까지 총 3045개 라벨링.
   - '레포 버킷' 처리 완료. 현재까지 총 3432개 라벨링.
   - '조직 버킷' 처리 완료. 현재까지 총 3706개 라벨링.

--- 작업 완료 ---
✅ 모든 라벨링 규칙이 적용된 결과가 아래 경로에 저장되었습니다.
   C:\Users\EL040\Desktop\Project\data\labeling_sheet_labeled_final.csv

[최종 'label' 컬럼 요약]
  - 0 (정상): 3794개
  - 1 (비정상): 3706개
